In [1]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from scipy import sparse
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel

In [2]:

train = pd.read_csv('../input/train2.csv')
test = pd.read_csv('../input/test2.csv')
test_id = pd.read_csv('../input/new_data/test_set.csv')[["id"]].copy()
y=(train["class"]-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
# trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
# trn_term_word = sparse.load_npz('../feature/text_tifi/trn_term_word.npz')
# trn_term_word_three = sparse.load_npz('../feature/text_tifi/trn_term_word_three.npz')
# train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three]).tocsr()
# del trn_term_doc,trn_term_word,trn_term_word_three
# import gc
# gc.collect()
# trn_term_word_four = sparse.load_npz('../feature/text_tifi/trn_term_word_four.npz')
# # trn_term_word_five = sparse.load_npz('../feature/text_tifi/trn_term_word_five.npz')
# trn_char = sparse.load_npz('../feature/text_tifi/trn_char.npz')
# docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
# docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [4]:
import gc
del train, test
gc.collect()

14

In [5]:
from keras.utils import np_utils
one_y_train=np_utils.to_categorical(y,num_classes=20)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# train_feaure =  hstack([train_feaure, trn_term_word_four,trn_char,doclen_word]).tocsr()
# del trn_term_word_four,trn_char,doclen_word
# gc.collect()

In [7]:
# train_feaure.shape
import pickle
svd_dir = '../feature/svd_5.pkl'
tifi_dir = '../feature/tifi_min4.pkl'
train_svd, test_svd= pickle.load(open(svd_dir,'rb'))
trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))
lda1 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature10.csv')
lda2 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature20.csv')
lda3 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature30.csv')
lda4 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature40.csv')
test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
train_feaure = hstack((trn_term_doc, train_svd, lda1.iloc[:102277,:], lda2.iloc[:102277,:], lda3.iloc[:102277,:], lda4.iloc[:102277,:], trn_term_doc)).tocsr()
test_festure = hstack((test_term_doc, test_svd, lda1.iloc[102277:,:], lda2.iloc[102277:,:], lda3.iloc[102277:,:], lda4.iloc[102277:,:], test_term_doc)).tocsr()

In [8]:
# test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
# test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')
# test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')
# test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')
# # test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')
# test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')
# test_festure =  hstack([test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test]).tocsr()
# del test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test
# gc.collect()

In [9]:
test_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
train_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat, average='macro'), True

In [ ]:
for i in range(19):
    print(i)
    train_target = one_y_train[:,i]
    model = LogisticRegression(C=4, solver='sag')
    sfm = SelectFromModel(model, threshold=0.1)
    print(train_feaure.shape)
    train_sparse_matrix = sfm.fit_transform(train_feaure, train_target)
    print(train_sparse_matrix.shape)
    train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_sparse_matrix, train_target, test_size=0.05, random_state=144)
    test_sparse_matrix = sfm.transform(test_festure)
    d_train = lgb.Dataset(train_sparse_matrix, label=y_train)
    d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid)
    watchlist = [d_train, d_valid]
    params = {'learning_rate': 0.1,
              'application': 'binary',
              'num_leaves': 50,
              'max_depth': 6,
              'verbosity': -1,
              'metric': 'auc',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,}
    model = lgb.train(params,
                      feval=lgb_f1_score,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      verbose_eval=10,
                      early_stopping_rounds=200)
    train_prob['class_prob_%s'%(i+1)] = model.predict(valid_sparse_matrix)
    test_prob['class_prob_%s'%(i+1)] = model.predict(test_sparse_matrix)

0
(102277, 800105)


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


(102277, 290513)
